In [1]:
# git clone https://github.com/Open-Catalyst-Project/ocp.git
# cd ocp
# pip install -e .

# pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
# pip install lmdb ase pymatgen
# pip install wandb tensorboard

# git clone https://github.com/SUNCAT-Center/CatKit.git
# cd CatKit
# SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True
# pip install -e .

In [2]:
# # https://nbviewer.org/gist/mhoffman/c418acb6b3f928eb4ef71f4001d120d9

# import requests
# import pprint
# import sys
# import string
# import json
# import io
# import copy
# import ase.io
# import ase.calculators.singlepoint
# import pickle

# GRAPHQL = 'http://api.catalysis-hub.org/graphql'

# def fetch(query):
#     return requests.get(
#         GRAPHQL, {'query': query}
#     ).json()['data']

# def reactions_from_dataset(pub_id, page_size=10):
#     reactions = []
#     has_next_page = True
#     start_cursor = ''
#     page = 0
#     while has_next_page:
#         data = fetch("""{{
#       reactions(pubId: "{pub_id}", first: {page_size}, after: "{start_cursor}") {{
#         totalCount
#         pageInfo {{
#           hasNextPage
#           hasPreviousPage
#           startCursor
#           endCursor 
#         }}  
#         edges {{
#           node {{
#             chemicalComposition
#             surfaceComposition
#             facet
#             sites
#             coverages
#             reactants
#             products
#             Equation
#             reactionEnergy
#             activationEnergy
#             dftCode
#             dftFunctional
#             username
#             pubId
#             reactionSystems {{
#               name
#               systems {{
#                 energy
#                 InputFile(format: "json")
#               }}
#             }}  
#           }}  
#         }}  
#       }}    
#     }}""".format(start_cursor=start_cursor,
#                  page_size=page_size,
#                  pub_id=pub_id,
#                 ))
#         has_next_page = data['reactions']['pageInfo']['hasNextPage']
#         start_cursor = data['reactions']['pageInfo']['endCursor']
#         page += 1
#         print(has_next_page, start_cursor, page_size * page, data['reactions']['totalCount'])
#         reactions.extend(map(lambda x: x['node'], data['reactions']['edges']))

#     return reactions

# def aseify_reactions(reactions):
#     for i, reaction in enumerate(reactions):
#         for j, _ in enumerate(reactions[i]['reactionSystems']):
#             with io.StringIO() as tmp_file:
#                 system = reactions[i]['reactionSystems'][j].pop('systems')
#                 tmp_file.write(system.pop('InputFile'))
#                 tmp_file.seek(0)
#                 atoms = ase.io.read(tmp_file, format='json')
#             calculator = ase.calculators.singlepoint.SinglePointCalculator(
#                 atoms,
#                 energy=system.pop('energy')
#             )
#             atoms.set_calculator(calculator)
#             #print(atoms.get_potential_energy())
#             reactions[i]['reactionSystems'][j]['atoms'] = atoms
#         # flatten list further into {name: atoms, ...} dictionary
#         reactions[i]['reactionSystems'] = {x['name']: x['atoms']
#                                           for x in reactions[i]['reactionSystems']}
        
# if __name__=='__main__':
#     raw_reactions = reactions_from_dataset("MamunHighT2019") # this will get all 88587 reactions
#     # raw_reactions = reactions_from_dataset("FesterEdge2017")
    
#     reactions = copy.deepcopy(raw_reactions)
#     aseify_reactions(reactions)
    
#     # Save reactions to a file
#     with open('/curdir/datasets/reactions.pickle', 'wb') as f:
#         pickle.dump(reactions, f)
    
#     # # Load reactions from a file
#     # with open('/curdir/datasets/reactions.pickle', 'rb') as f:
#     #     loaded_reactions = pickle.load(f)
    

In [3]:
# !pip install ray

In [4]:
import pickle
import os
import numpy as np
import ase.io
from ase.constraints import FixAtoms
from ase.build import add_adsorbate, molecule, surface
from pymatgen.ext.matproj import MPRester
from pymatgen.core.surface import generate_all_slabs, SlabGenerator
from pymatgen.io.ase import AseAtomsAdaptor
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
import json
import ray
import pandas as pd
from transformers import AutoTokenizer, AutoModel, pipeline

2023-10-17 20:01:10,400	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
with open('datasets/reactions.pickle', 'rb') as f:
    loaded_reactions = pickle.load(f)

In [6]:
len(loaded_reactions)

88587

In [7]:
loaded_reactions[4000:4001]

[{'chemicalComposition': 'Re3V9',
  'surfaceComposition': 'V3Re',
  'facet': '111',
  'sites': '{"C": "hollow", "O": "hollow"}',
  'coverages': '{"C": 0.25, "O": 0.25}',
  'reactants': '{"star": 3, "CO2gas": 1}',
  'products': '{"Cstar": 1, "Ostar": 2}',
  'Equation': 'CO2(g) + 3* -> C* + 2O*',
  'reactionEnergy': -7.092022703231123,
  'activationEnergy': None,
  'dftCode': 'Quantum ESPRESSO 5.1',
  'dftFunctional': 'BEEF-vdW',
  'username': 'winther@stanford.edu',
  'pubId': 'MamunHighT2019',
  'reactionSystems': {'bulk': Atoms(symbols='V3Re', pbc=True, cell=[3.847775727704837, 3.847775727704837, 3.847775727704837], calculator=SinglePointCalculator(...)),
   'star': Atoms(symbols='V3ReV3ReV3Re', pbc=True, cell=[[5.441575614916559, 0.0, 0.0], [-2.7207878074582794, 4.712542666507961, 0.0], [0.0, 0.0, 24.443027270242712]], calculator=SinglePointCalculator(...)),
   'Ostar': Atoms(symbols='VReV3ReV3ReV2O', pbc=True, cell=[[5.441575614916559, 0.0, 0.0], [2.7207878074582794, 4.7125426665079

In [8]:
# m = MPRester('Yct0KDbJbqMLWluZEovkwrLXh2VRHXbc')

In [9]:
ray.init(ignore_reinit_error=True)

2023-10-17 20:01:37,989	INFO worker.py:1621 -- Started a local Ray instance.


Python version:,3.10.9
Ray version:,2.6.2


In [10]:
config_yml_path = "ocp/configs/is2re/all/dimenet_plus_plus/dpp.yml"
checkpoint_path = "dimenetpp_all.pt"

# Define the calculator
calc_id = ray.put(OCPCalculator(config_yml=config_yml_path, checkpoint=checkpoint_path))

RuntimeError: !atom.empty() INTERNAL ASSERT FAILED at "/opt/conda/conda-bld/pytorch_1678402411778/work/aten/src/ATen/core/qualified_name.h":24, please report a bug to PyTorch. Invalid name for qualified name: '__torch__.ocpmodels.models..equiformer_v2.gaussian_rbf.gaussian'

In [10]:
def get_energy(ase_slab, ads_symbol):
    
    adslab = ase_slab.copy()
    adslab.calc = ray.get(calc_id)
    # print(adslab)
    
    slab_energy = adslab.get_potential_energy()
    
    # add the molecule
    adsorbate = molecule(ads_symbol)
    add_adsorbate(adslab, adsorbate, 3, offset=(1, 1))
    # print(adslab)
    
    ads_energy = adslab.get_potential_energy()
    
    # because this is a fingerprint we do not need to reference to gas molecules 
    # because this is the same constant across surfaces
    
    return ads_energy - slab_energy

In [11]:
get_energy(loaded_reactions[5]['reactionSystems']['star'], 'O2' )

3.444236993789673

In [14]:
from pprint import pprint
for i in range(1):
    pprint(loaded_reactions[i])

{'Equation': '0.5N2(g) + * -> N*',
 'activationEnergy': None,
 'chemicalComposition': 'Sc3V9',
 'coverages': '{"N": 0.25}',
 'dftCode': 'Quantum ESPRESSO 5.1',
 'dftFunctional': 'BEEF-vdW',
 'facet': '111',
 'products': '{"Nstar": 1}',
 'pubId': 'MamunHighT2019',
 'reactants': '{"star": 1, "N2gas": 0.5}',
 'reactionEnergy': -2.810391181412342,
 'reactionSystems': {'N2gas': Atoms(symbols='N2', pbc=True, cell=[19.999998584662084, 19.999998584662084, 21.129978504696908], calculator=SinglePointCalculator(...)),
                     'Nstar': Atoms(symbols='VScV3ScV3ScV2N', pbc=True, cell=[[5.6401896008612615, 0.0, 0.0], [2.8200948004306308, 4.884546654335772, 0.0], [0.0, 0.0, 24.60519325876673]], calculator=SinglePointCalculator(...)),
                     'bulk': Atoms(symbols='V3Sc', pbc=True, cell=[3.9882157177663338, 3.9882157177663338, 3.9882157177663338], calculator=SinglePointCalculator(...)),
                     'star': Atoms(symbols='V3ScV3ScV3Sc', pbc=True, cell=[[5.6401896008612

In [30]:
list_adsorbate = ['H', 'O', 'C', 'CO', 'OH', 'N2', 'CO2', 'NH3', 'CH4'] ## 'H2S' 

In [17]:
@ray.remote
def process_reaction(react, list_adsorbate):
    try:
        d_prod = json.loads(react['products'])
        if len(d_prod.keys()) != 1:
            return None

        sc = react['surfaceComposition']
        facet = react['facet']
        star = react['reactionSystems']['star']
        list_sdesc = []
        ase_slab = star
        for ads_symbol in list_adsorbate:
            sdesc = get_energy(ase_slab, ads_symbol)
            list_sdesc.append(sdesc)

        pkey = list(d_prod.keys())[0].replace('star', '')
        pval = d_prod[list(d_prod.keys())[0]]
        pdesc = []

        re = react['reactionEnergy']
        nre = re/pval
        
        eqn = react['Equation']
        ae = react['activationEnergy']
        cc = react['chemicalComposition']
        d_cvr = json.loads(react['coverages'])
        dft_code = react['dftCode']
        dft_func = react['dftFunctional']
        pubid = react['pubId']
        d_reactants = json.loads(react['reactants'])
        sites = json.loads(react['sites'])
        username = react['username']

        rinfo = {
            'sc': sc,
            'facet': facet,
            'star': str(star.symbols),
            'sdesc': list_sdesc,
            
            'pkey': pkey,
            'pval': pval,
            'pdesc': pdesc,
            
            're': re,
            'nre': nre,
            
            'd_prod': d_prod,
            'eqn': eqn,
            'ae': ae, 
            'cc': cc, 
            'd_cvr': d_cvr, 
            'dft_code': dft_code,
            'dft_func': dft_func,
            'pubid': pubid,
            'd_reactants': d_reactants,
            'sites': sites,
            'username': username
            
        }
        return rinfo
    except Exception as e:
        return None
    
futures = [process_reaction.remote(react, list_adsorbate) for react in loaded_reactions[:]]
results = ray.get(futures)

# Filter out None values
list_rinfo = [r for r in results if r is not None]

In [19]:
print(len(list_rinfo))
print(list_rinfo[0])

45136
{'sc': 'V3Sc', 'facet': '111', 'star': 'V3ScV3ScV3Sc', 'sdesc': [0.5108659267425537, -0.4489332437515259, -0.230901837348938, -0.4028477668762207, 0.3514571189880371, 0.1492377519607544, -0.04714500904083252, 0.31800568103790283, -1.1302334070205688], 'pkey': 'N', 'pval': 1, 'pdesc': [], 're': -2.810391181412342, 'nre': -2.810391181412342, 'd_prod': {'Nstar': 1}, 'eqn': '0.5N2(g) + * -> N*', 'ae': None, 'cc': 'Sc3V9', 'd_cvr': {'N': 0.25}, 'dft_code': 'Quantum ESPRESSO 5.1', 'dft_func': 'BEEF-vdW', 'pubid': 'MamunHighT2019', 'd_reactants': {'star': 1, 'N2gas': 0.5}, 'sites': {'N': 'hollow|A_A_A|HCP'}, 'username': 'winther@stanford.edu'}


In [122]:
# with open('datasets/reactions_info.pickle', 'wb') as f:
#     pickle.dump(list_rinfo, f)

In [13]:
with open('datasets/reactions_info.pickle', 'rb') as f:
    loaded_list_rinfo = pickle.load(f)

In [14]:
print(len(loaded_list_rinfo))
print(loaded_list_rinfo[0])

45136
{'sc': 'V3Sc', 'facet': '111', 'star': 'V3ScV3ScV3Sc', 'sdesc': [0.5108659267425537, -0.4489332437515259, -0.230901837348938, -0.4028477668762207, 0.3514571189880371, 0.1492377519607544, -0.04714500904083252, 0.31800568103790283, -1.1302334070205688], 'pkey': 'N', 'pval': 1, 'pdesc': [], 're': -2.810391181412342, 'nre': -2.810391181412342, 'd_prod': {'Nstar': 1}, 'eqn': '0.5N2(g) + * -> N*', 'ae': None, 'cc': 'Sc3V9', 'd_cvr': {'N': 0.25}, 'dft_code': 'Quantum ESPRESSO 5.1', 'dft_func': 'BEEF-vdW', 'pubid': 'MamunHighT2019', 'd_reactants': {'star': 1, 'N2gas': 0.5}, 'sites': {'N': 'hollow|A_A_A|HCP'}, 'username': 'winther@stanford.edu'}


In [15]:
# for d in loaded_list_rinfo[4000:4001]:
#     pprint(d)

In [16]:
## todo: generate product descriptions

In [17]:
list_pkey =  []
for rinfo in loaded_list_rinfo:
    pkey = rinfo['pkey']
    list_pkey.append(pkey)

In [18]:
list(set(list_pkey))

['H', 'C', 'CH3', 'O', 'CH2', 'S', 'SH', 'NH', 'OH', 'H2O', 'N', 'CH']

In [19]:
d_pkey_vs_smiles = {
    ## pubchem: one entry/ multi entry/ previous datasets
    'S': '[S]',
    'O': '[O]',
    'CH': '[CH]', 
    'H': '[H]',
    'SH': '[SH]',    
    'NH': '[NH]',
    'CH3': '[CH3]',   
    'H2O': 'O',
    'C': '[C]',   
    'N': '[N]',    
    'OH': '[OH]',
    'CH2': '[CH2]', 
}

In [20]:
## pretrained model
tokenizer = AutoTokenizer.from_pretrained("mrm8488/chEMBL26_smiles_v2")
model = AutoModel.from_pretrained("mrm8488/chEMBL26_smiles_v2")
fe = pipeline('feature-extraction', model=model, tokenizer=tokenizer, device=-1) ## device= (0 for GPU, -1 for CPU)

Some weights of the model checkpoint at mrm8488/chEMBL26_smiles_v2 were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
list_pkey= list(d_pkey_vs_smiles.keys())
list_smiles= list(d_pkey_vs_smiles.values())
list_emb_1 = fe(list_smiles)

In [22]:
list_emb_2 = []
for emb_1 in list_emb_1:
    emb_2 = np.mean(np.array(emb_1[0]), axis=0)
    list_emb_2.append(emb_2)
arr_emb = np.array(list_emb_2)
print(arr_emb.shape)

(12, 768)


In [23]:
d_pkey_vs_pdesc = {}
for i, pkey in enumerate(list_pkey):
    pdesc = arr_emb[i]
    d_pkey_vs_pdesc[pkey] = pdesc    

In [24]:
for irinfo, rinfo in enumerate(loaded_list_rinfo):
    pkey = rinfo['pkey']
    pdesc = list(d_pkey_vs_pdesc[pkey])
    rinfo['pdesc'] = pdesc
    if irinfo%10000==0:
        print(irinfo)

0
10000
20000
30000
40000


In [25]:
# Now, to convert this into a DataFrame:
df = pd.DataFrame(loaded_list_rinfo)

In [26]:
df.head(2)

,sc,facet,star,sdesc,pkey,pval,pdesc,re,nre,d_prod,eqn,ae,cc,d_cvr,dft_code,dft_func,pubid,d_reactants,sites,username
0,V3Sc,111,V3ScV3ScV3Sc,"[0.5108659267425537, -0.4489332437515259, -0.2...",N,1,"[0.48027973622083664, -0.6266024529933929, 0.8...",-2.810391,-2.810391,{'Nstar': 1},0.5N2(g) + * -> N*,None,Sc3V9,{'N': 0.25},Quantum ESPRESSO 5.1,BEEF-vdW,MamunHighT2019,"{'star': 1, 'N2gas': 0.5}",{'N': 'hollow|A_A_A|HCP'},winther@stanford.edu
1,HgTa,101,Ta2Hg2Ta2Hg2Ta2Hg2,"[0.34193313121795654, 0.4899855852127075, 0.61...",O,1,"[0.42940246015787126, -0.6335948944091797, 0.9...",-2.099597,-2.099597,{'Ostar': 1},H2O(g) - H2(g) + * -> O*,None,Hg6Ta6,{'O': 0.25},Quantum ESPRESSO 5.1,BEEF-vdW,MamunHighT2019,"{'star': 1, 'H2gas': -1.0, 'H2Ogas': 1.0}",{'O': 'bridge|B_B|A'},winther@stanford.edu


In [27]:
print(df.shape)

(45136, 20)


In [28]:
df = df.sort_values(by='nre')
df = df.groupby(['star', 'pkey']).first().reset_index()
print(df.shape)

(11257, 20)


In [31]:
list_scols = ['s' + str(i) for i in range(len(list_adsorbate))]

In [32]:
df[list_scols] = pd.DataFrame(df.sdesc.tolist(), index= df.index)

In [33]:
print(df.shape)

(11257, 29)


In [34]:
list_pcols = ['p' + str(i) for i in range(int(arr_emb[0].shape[0]))]

In [35]:
try:
    df[list_pcols] = pd.DataFrame(df.pdesc.tolist(), index= df.index)
except Exception as e:
    print(e)

In [37]:
print(df.shape)

(11257, 797)


In [38]:
df.head(2)

,star,pkey,sc,facet,sdesc,pval,pdesc,re,nre,d_prod,...,p758,p759,p760,p761,p762,p763,p764,p765,p766,p767
0,Ag12,C,Ag,111,"[1.3833613395690918, 2.2273499965667725, 3.257...",1,"[1.5289912700653077, -0.41977829933166505, -0....",5.608979,5.608979,{'Cstar': 1},...,-0.516870,1.128245,0.940684,1.901682,-0.801239,-1.315113,-0.644158,0.311038,-0.162647,-0.846766
1,Ag12,CH,Ag,111,"[1.3833613395690918, 2.2273499965667725, 3.257...",1,"[1.423767828196287, 0.0374512255191803, 0.2759...",3.996914,3.996914,{'CHstar': 1},...,-1.233719,0.248250,0.536033,1.534562,-0.736114,-0.747634,-1.661336,0.266347,-0.284854,-0.232635


In [39]:
# df.to_pickle('datasets/reactions_info_df.pickle')

In [40]:
df = pd.read_pickle('datasets/reactions_info_df.pickle')

In [41]:
print(df.shape)

(11257, 797)


In [42]:
df = df[list_scols + list_pcols + ['nre']]

In [43]:
df.shape

(11257, 778)

In [44]:
df.head(2)

,s0,s1,s2,s3,s4,s5,s6,s7,s8,p0,...,p759,p760,p761,p762,p763,p764,p765,p766,p767,nre
0,1.383361,2.22735,3.257025,1.991915,2.301531,2.101231,2.811456,0.432559,-1.222454,1.528991,...,1.128245,0.940684,1.901682,-0.801239,-1.315113,-0.644158,0.311038,-0.162647,-0.846766,5.608979
1,1.383361,2.22735,3.257025,1.991915,2.301531,2.101231,2.811456,0.432559,-1.222454,1.423768,...,0.248250,0.536033,1.534562,-0.736114,-0.747634,-1.661336,0.266347,-0.284854,-0.232635,3.996914


In [45]:
# df.to_pickle('datasets/reactions_info_df_dataset.pickle')

In [46]:
df = pd.read_pickle('datasets/reactions_info_df_dataset.pickle')

In [47]:
df.shape

(11257, 778)